#### loading data

In [31]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine
import networkx as nx
import matplotlib.pyplot as plt

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CRM/Homework 07/dataset.csv")

In [34]:
df.head()

,Unnamed: 0,จักรยานเสือภูเขา,คอลลาเจน,Brownie อวกาศ,แม่โขง,กัญชา,กัญชง,เครื่องวัดความดัน,หลอดไฟเปลี่ยนสีได้,คอร์ส bikini wax,เก้าอี้นวดไฟฟ้า,ต้นไม้ประดับห้อง,หมอนข้าง,ตาชั่ง,คอร์สโบท็อกกราม,แสงโสม,กะหรี่(ปั๊บ),Lego,แฟน ที่แปลว่าพัดลม,ยาดอง,ชุดนอนไม่ได้นอน,ที่ยกน้ำหนัก (Dumbell),เครื่องดูดฝุ่นอัตโนมัติ (robot),เนื้อวากิล,หมูแดดเดียว,drone,เครื่องย่อยเศษอาหาร,เสื้อซีทรู,ไพ่ยิปซี,เนื้อย่าง,เครื่องทำลายเอกสาร,กระเป๋า,ชาบู,Surfskate,Board game,ชุดน้องหมา,Omakase,เม็ดมะม่วงหิมพานต์,ไฟอัจฉริยะ,บัตตาเลี่ยนที่ตัดผม,ไดร์เป่าผม Dyson,แผ่นเสียง,กรรไกรตัดขนจมูก,PS5,เครื่องตรวจน้ำตาลในเลือด,โรตี,กะปิดอง,น้องหมา,แพ็คเกจไทยเที่ยวไทย,เกมส์ FIFA Online,มะม่วงเบา,ห้องน้ำแมวอัตโนมัติ,กระเป๋า chanel,ข้าวขาหมูคากิคาจักออนไลน์,ถั่งเช่า,หมอนสุขภาพสำหรับคนนอนไม่หลับ,อาหารแมวคลีนเพื่อสุขภาพลดหุ่น,น้ำส้มดีโด้,Software ถูกลิขสิทธิ์,ครีมกันแดด
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,0,1,0,1,1,0,1,0,1,1,0,0,0,0,1,1,0.0,1,0,0,0.0,0,0,0,0,0,0.0,0,0,1,1,1
1,1,0,1,0,1,0,0,1,0,0,0,0,1,1,0,1,1,0,1,0,1,1,1,0,1,0,0,1,1,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,1.0,1,1,0,1.0,0,1,0,0,0,1.0,1,0,1,0,1
2,2,1,0,0,0,0,0,1,1,0,0,1,1,1,0,0,1,1,1,0,1,1,0,1,1,1,0,1,0,1,1,1,1,1,1,1,0,1,1,1,0,0,0,0,1.0,1,0,1,0.0,0,1,0,0,0,0.0,1,0,1,1,1
3,3,0,0,0,1,1,0,0,0,0,1,1,1,1,0,1,1,1,1,1,0,0,1,1,1,0,0,0,0,1,0,1,1,0,1,0,1,1,0,1,0,0,1,0,0.0,1,0,0,0.0,1,0,0,0,0,0.0,0,0,0,0,1
4,4,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,1,1,0,1,0,0,0,1,0,0,1,1,0,1,1,0,1,0,0,1,0,0,0,0.0,1,0,1,NaN,0,0,0,1,0,0.0,1,0,1,1,1


In [35]:
df.drop(df.columns[0], axis=1,inplace = True)

In [36]:
df.head()

,จักรยานเสือภูเขา,คอลลาเจน,Brownie อวกาศ,แม่โขง,กัญชา,กัญชง,เครื่องวัดความดัน,หลอดไฟเปลี่ยนสีได้,คอร์ส bikini wax,เก้าอี้นวดไฟฟ้า,ต้นไม้ประดับห้อง,หมอนข้าง,ตาชั่ง,คอร์สโบท็อกกราม,แสงโสม,กะหรี่(ปั๊บ),Lego,แฟน ที่แปลว่าพัดลม,ยาดอง,ชุดนอนไม่ได้นอน,ที่ยกน้ำหนัก (Dumbell),เครื่องดูดฝุ่นอัตโนมัติ (robot),เนื้อวากิล,หมูแดดเดียว,drone,เครื่องย่อยเศษอาหาร,เสื้อซีทรู,ไพ่ยิปซี,เนื้อย่าง,เครื่องทำลายเอกสาร,กระเป๋า,ชาบู,Surfskate,Board game,ชุดน้องหมา,Omakase,เม็ดมะม่วงหิมพานต์,ไฟอัจฉริยะ,บัตตาเลี่ยนที่ตัดผม,ไดร์เป่าผม Dyson,แผ่นเสียง,กรรไกรตัดขนจมูก,PS5,เครื่องตรวจน้ำตาลในเลือด,โรตี,กะปิดอง,น้องหมา,แพ็คเกจไทยเที่ยวไทย,เกมส์ FIFA Online,มะม่วงเบา,ห้องน้ำแมวอัตโนมัติ,กระเป๋า chanel,ข้าวขาหมูคากิคาจักออนไลน์,ถั่งเช่า,หมอนสุขภาพสำหรับคนนอนไม่หลับ,อาหารแมวคลีนเพื่อสุขภาพลดหุ่น,น้ำส้มดีโด้,Software ถูกลิขสิทธิ์,ครีมกันแดด
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,0,1,0,1,1,0,1,0,1,1,0,0,0,0,1,1,0.0,1,0,0,0.0,0,0,0,0,0,0.0,0,0,1,1,1
1,0,1,0,1,0,0,1,0,0,0,0,1,1,0,1,1,0,1,0,1,1,1,0,1,0,0,1,1,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,1.0,1,1,0,1.0,0,1,0,0,0,1.0,1,0,1,0,1
2,1,0,0,0,0,0,1,1,0,0,1,1,1,0,0,1,1,1,0,1,1,0,1,1,1,0,1,0,1,1,1,1,1,1,1,0,1,1,1,0,0,0,0,1.0,1,0,1,0.0,0,1,0,0,0,0.0,1,0,1,1,1
3,0,0,0,1,1,0,0,0,0,1,1,1,1,0,1,1,1,1,1,0,0,1,1,1,0,0,0,0,1,0,1,1,0,1,0,1,1,0,1,0,0,1,0,0.0,1,0,0,0.0,1,0,0,0,0,0.0,0,0,0,0,1
4,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,1,1,0,1,0,0,0,1,0,0,1,1,0,1,1,0,1,0,0,1,0,0,0,0.0,1,0,1,NaN,0,0,0,1,0,0.0,1,0,1,1,1


#### mock up user_ida

In [37]:
len(df)

45

In [38]:
df_user_id = pd.DataFrame(list(range(1,46)),columns=["User_ID"])

In [39]:
data = pd.concat([df_user_id, df], axis=1)

In [79]:
data.head()

,User_ID,จักรยานเสือภูเขา,คอลลาเจน,Brownie อวกาศ,แม่โขง,กัญชา,กัญชง,เครื่องวัดความดัน,หลอดไฟเปลี่ยนสีได้,คอร์ส bikini wax,เก้าอี้นวดไฟฟ้า,ต้นไม้ประดับห้อง,หมอนข้าง,ตาชั่ง,คอร์สโบท็อกกราม,แสงโสม,กะหรี่(ปั๊บ),Lego,แฟน ที่แปลว่าพัดลม,ยาดอง,ชุดนอนไม่ได้นอน,ที่ยกน้ำหนัก (Dumbell),เครื่องดูดฝุ่นอัตโนมัติ (robot),เนื้อวากิล,หมูแดดเดียว,drone,เครื่องย่อยเศษอาหาร,เสื้อซีทรู,ไพ่ยิปซี,เนื้อย่าง,เครื่องทำลายเอกสาร,กระเป๋า,ชาบู,Surfskate,Board game,ชุดน้องหมา,Omakase,เม็ดมะม่วงหิมพานต์,ไฟอัจฉริยะ,บัตตาเลี่ยนที่ตัดผม,ไดร์เป่าผม Dyson,แผ่นเสียง,กรรไกรตัดขนจมูก,PS5,เครื่องตรวจน้ำตาลในเลือด,โรตี,กะปิดอง,น้องหมา,แพ็คเกจไทยเที่ยวไทย,เกมส์ FIFA Online,มะม่วงเบา,ห้องน้ำแมวอัตโนมัติ,กระเป๋า chanel,ข้าวขาหมูคากิคาจักออนไลน์,ถั่งเช่า,หมอนสุขภาพสำหรับคนนอนไม่หลับ,อาหารแมวคลีนเพื่อสุขภาพลดหุ่น,น้ำส้มดีโด้,Software ถูกลิขสิทธิ์,ครีมกันแดด
0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,0,1,0,1,1,0,1,0,1,1,0,0,0,0,1,1,0.0,1,0,0,0.0,0,0,0,0,0,0.0,0,0,1,1,1
1,2,0,1,0,1,0,0,1,0,0,0,0,1,1,0,1,1,0,1,0,1,1,1,0,1,0,0,1,1,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,1.0,1,1,0,1.0,0,1,0,0,0,1.0,1,0,1,0,1
2,3,1,0,0,0,0,0,1,1,0,0,1,1,1,0,0,1,1,1,0,1,1,0,1,1,1,0,1,0,1,1,1,1,1,1,1,0,1,1,1,0,0,0,0,1.0,1,0,1,0.0,0,1,0,0,0,0.0,1,0,1,1,1
3,4,0,0,0,1,1,0,0,0,0,1,1,1,1,0,1,1,1,1,1,0,0,1,1,1,0,0,0,0,1,0,1,1,0,1,0,1,1,0,1,0,0,1,0,0.0,1,0,0,0.0,1,0,0,0,0,0.0,0,0,0,0,1
4,5,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,1,1,0,1,0,0,0,1,0,0,1,1,0,1,1,0,1,0,0,1,0,0,0,0.0,1,0,1,0.0,0,0,0,1,0,0.0,1,0,1,1,1


In [41]:
product_list = df.columns
product_list

Index(['จักรยานเสือภูเขา', 'คอลลาเจน', 'Brownie อวกาศ', 'แม่โขง', 'กัญชา',
       'กัญชง', 'เครื่องวัดความดัน', 'หลอดไฟเปลี่ยนสีได้', 'คอร์ส bikini wax',
       'เก้าอี้นวดไฟฟ้า', 'ต้นไม้ประดับห้อง', 'หมอนข้าง', 'ตาชั่ง ',
       'คอร์สโบท็อกกราม', 'แสงโสม', 'กะหรี่(ปั๊บ)', 'Lego',
       'แฟน ที่แปลว่าพัดลม', 'ยาดอง', 'ชุดนอนไม่ได้นอน',
       'ที่ยกน้ำหนัก (Dumbell)', 'เครื่องดูดฝุ่นอัตโนมัติ (robot)',
       'เนื้อวากิล', 'หมูแดดเดียว', 'drone', 'เครื่องย่อยเศษอาหาร',
       'เสื้อซีทรู', 'ไพ่ยิปซี', 'เนื้อย่าง', 'เครื่องทำลายเอกสาร', 'กระเป๋า',
       'ชาบู', 'Surfskate', 'Board game', 'ชุดน้องหมา', 'Omakase',
       'เม็ดมะม่วงหิมพานต์', 'ไฟอัจฉริยะ', 'บัตตาเลี่ยนที่ตัดผม',
       'ไดร์เป่าผม Dyson', 'แผ่นเสียง', 'กรรไกรตัดขนจมูก', 'PS5',
       'เครื่องตรวจน้ำตาลในเลือด', 'โรตี', 'กะปิดอง', 'น้องหมา',
       'แพ็คเกจไทยเที่ยวไทย', 'เกมส์ FIFA Online', 'มะม่วงเบา',
       'ห้องน้ำแมวอัตโนมัติ', 'กระเป๋า chanel', 'ข้าวขาหมูคากิคาจักออนไลน์',
       'ถั่งเช่า', 'หมอนสุขภาพสำหรับคนนอ

#### Preprocessing

In [42]:
user_id = list(range(1,46))

In [43]:
data.fillna(0,inplace=True)

In [44]:
data.astype("int64").dtypes

User_ID                            int64
จักรยานเสือภูเขา                   int64
คอลลาเจน                           int64
Brownie อวกาศ                      int64
แม่โขง                             int64
กัญชา                              int64
กัญชง                              int64
เครื่องวัดความดัน                  int64
หลอดไฟเปลี่ยนสีได้                 int64
คอร์ส bikini wax                   int64
เก้าอี้นวดไฟฟ้า                    int64
ต้นไม้ประดับห้อง                   int64
หมอนข้าง                           int64
ตาชั่ง                             int64
คอร์สโบท็อกกราม                    int64
แสงโสม                             int64
กะหรี่(ปั๊บ)                       int64
Lego                               int64
แฟน ที่แปลว่าพัดลม                 int64
ยาดอง                              int64
ชุดนอนไม่ได้นอน                    int64
ที่ยกน้ำหนัก (Dumbell)             int64
เครื่องดูดฝุ่นอัตโนมัติ (robot)    int64
เนื้อวากิล                         int64
หมูแดดเดียว     

In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 60 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   User_ID                          45 non-null     int64  
 1   จักรยานเสือภูเขา                 45 non-null     int64  
 2   คอลลาเจน                         45 non-null     int64  
 3   Brownie อวกาศ                    45 non-null     int64  
 4   แม่โขง                           45 non-null     int64  
 5   กัญชา                            45 non-null     int64  
 6   กัญชง                            45 non-null     int64  
 7   เครื่องวัดความดัน                45 non-null     int64  
 8   หลอดไฟเปลี่ยนสีได้               45 non-null     int64  
 9   คอร์ส bikini wax                 45 non-null     int64  
 10  เก้าอี้นวดไฟฟ้า                  45 non-null     int64  
 11  ต้นไม้ประดับห้อง                 45 non-null     int64  
 12  หมอนข้าง                

In [46]:
rows_list = []
for prod in product_list:
    # print(prod)
    for id in user_id:
        value = int(data[data["User_ID"] == id][prod].values)
        # print(value)
        new_row = {'user_id':id,"product" : prod , 'rate':value}
        print(new_row)
        rows_list.append(new_row)

{'user_id': 1, 'product': 'จักรยานเสือภูเขา', 'rate': 0}
{'user_id': 2, 'product': 'จักรยานเสือภูเขา', 'rate': 0}
{'user_id': 3, 'product': 'จักรยานเสือภูเขา', 'rate': 1}
{'user_id': 4, 'product': 'จักรยานเสือภูเขา', 'rate': 0}
{'user_id': 5, 'product': 'จักรยานเสือภูเขา', 'rate': 0}
{'user_id': 6, 'product': 'จักรยานเสือภูเขา', 'rate': 1}
{'user_id': 7, 'product': 'จักรยานเสือภูเขา', 'rate': 0}
{'user_id': 8, 'product': 'จักรยานเสือภูเขา', 'rate': 0}
{'user_id': 9, 'product': 'จักรยานเสือภูเขา', 'rate': 0}
{'user_id': 10, 'product': 'จักรยานเสือภูเขา', 'rate': 1}
{'user_id': 11, 'product': 'จักรยานเสือภูเขา', 'rate': 0}
{'user_id': 12, 'product': 'จักรยานเสือภูเขา', 'rate': 1}
{'user_id': 13, 'product': 'จักรยานเสือภูเขา', 'rate': 0}
{'user_id': 14, 'product': 'จักรยานเสือภูเขา', 'rate': 0}
{'user_id': 15, 'product': 'จักรยานเสือภูเขา', 'rate': 1}
{'user_id': 16, 'product': 'จักรยานเสือภูเขา', 'rate': 1}
{'user_id': 17, 'product': 'จักรยานเสือภูเขา', 'rate': 1}
{'user_id': 18, 'produc

In [47]:
product_data = pd.DataFrame(rows_list)   

In [48]:
product_data

,user_id,product,rate
0,1,จักรยานเสือภูเขา,0
1,2,จักรยานเสือภูเขา,0
2,3,จักรยานเสือภูเขา,1
3,4,จักรยานเสือภูเขา,0
4,5,จักรยานเสือภูเขา,0
...,...,...,...
2650,41,ครีมกันแดด,1
2651,42,ครีมกันแดด,1
2652,43,ครีมกันแดด,1
2653,44,ครีมกันแดด,1


#### Utility Matrix based on products and user rate

In [49]:
utility_matrix = product_data.pivot_table(values='rate', index='user_id', columns='product', fill_value=0)
utility_matrix.head()

product,Board game,Brownie อวกาศ,Lego,Omakase,PS5,Software ถูกลิขสิทธิ์,Surfskate,drone,กรรไกรตัดขนจมูก,กระเป๋า,กระเป๋า chanel,กะปิดอง,กะหรี่(ปั๊บ),กัญชง,กัญชา,ข้าวขาหมูคากิคาจักออนไลน์,ครีมกันแดด,คอร์ส bikini wax,คอร์สโบท็อกกราม,คอลลาเจน,จักรยานเสือภูเขา,ชาบู,ชุดนอนไม่ได้นอน,ชุดน้องหมา,ตาชั่ง,ต้นไม้ประดับห้อง,ถั่งเช่า,ที่ยกน้ำหนัก (Dumbell),น้องหมา,น้ำส้มดีโด้,บัตตาเลี่ยนที่ตัดผม,มะม่วงเบา,ยาดอง,หมอนข้าง,หมอนสุขภาพสำหรับคนนอนไม่หลับ,หมูแดดเดียว,หลอดไฟเปลี่ยนสีได้,ห้องน้ำแมวอัตโนมัติ,อาหารแมวคลีนเพื่อสุขภาพลดหุ่น,เกมส์ FIFA Online,เก้าอี้นวดไฟฟ้า,เครื่องดูดฝุ่นอัตโนมัติ (robot),เครื่องตรวจน้ำตาลในเลือด,เครื่องทำลายเอกสาร,เครื่องย่อยเศษอาหาร,เครื่องวัดความดัน,เนื้อย่าง,เนื้อวากิล,เม็ดมะม่วงหิมพานต์,เสื้อซีทรู,แผ่นเสียง,แพ็คเกจไทยเที่ยวไทย,แฟน ที่แปลว่าพัดลม,แม่โขง,แสงโสม,โรตี,ไดร์เป่าผม Dyson,ไพ่ยิปซี,ไฟอัจฉริยะ
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,0,1,1,1,1,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,1,1,0,0,0,1,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,1,0,0,1,0,1,1,0,1,0,1,1,0,1,0,1,0,1,1,1,0,0,0,0,0,1,1,0,0,1,0,0,1,1,0,1,1,1,1,1,0,1,0
3,1,0,1,0,0,1,1,1,0,1,0,0,1,0,0,0,1,0,0,0,1,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,0,0,0,0,0,1,1,0,1,1,1,1,1,0,0,1,0,0,1,0,0,1
4,1,0,1,1,0,0,0,0,1,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,1,1,0,1,0,0,0,1,1,1,0,0,0,0,1,1,1,0,0,0,1,1,1,1,0,0,0
5,1,0,0,0,0,1,0,0,0,1,1,0,1,0,0,0,1,0,1,1,0,1,0,1,0,1,0,1,1,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,1,0


#### Correlation-Based Recommender (User - User)

In [50]:
from sklearn.metrics.pairwise import cosine_similarity
import operator
def similar_users(user_id, matrix, k=3):
    # create a df of just the current user
    user = matrix[matrix.index == user_id]
    
    # and a df of all other users
    other_users = matrix[matrix.index != user_id]
    
    # calc cosine similarity between user and each other user
    similarities = cosine_similarity(user,other_users)[0].tolist()
    
    # create list of indices of these users
    indices = other_users.index.tolist()
    
    # create key/values pairs of user index and their similarity
    index_similarity = dict(zip(indices, similarities))
    
    # sort by similarity
    index_similarity_sorted = sorted(index_similarity.items(), key=operator.itemgetter(1))
    index_similarity_sorted.reverse()
    
    # grab k users off the top
    top_users_similarities = index_similarity_sorted[:k]
    users = [u[0] for u in top_users_similarities]
    
    return users

### Assume user 1 come to our store and would like to reccomend him based on other user in our database

Result shown that there are 3 user similar to user_id = 1 
so that he probably like below item

In [51]:
current_user= 1
similar_user_indices = similar_users(current_user, utility_matrix)
print(similar_user_indices)

[13, 4, 33]


In [52]:
def product_rec(current_user):
    similar_user_indices = similar_users(current_user, utility_matrix)
    user_sim = similar_user_indices
    tf1 = product_data[(product_data["user_id"] == user_sim[0] )& product_data["rate"] == 1]
    tf2=product_data[(product_data["user_id"] == user_sim[1] )& product_data["rate"] == 1]
    tf3 = product_data[(product_data["user_id"] == user_sim[2] )& product_data["rate"] == 1]
    user = product_data[product_data["user_id"] == current_user &(product_data["rate"]==1)]
    result_rec = tf1.append([tf2,tf3,user]).drop_duplicates('product').sort_index()
    return result_rec

In [53]:
product_rec(1)

,user_id,product,rate
138,4,แม่โขง,1
183,4,กัญชา,1
408,4,เก้าอี้นวดไฟฟ้า,1
453,4,ต้นไม้ประดับห้อง,1
498,4,หมอนข้าง,1
552,13,ตาชั่ง,1
633,4,แสงโสม,1
687,13,กะหรี่(ปั๊บ),1
723,4,Lego,1
777,13,แฟน ที่แปลว่าพัดลม,1


In [131]:
rows_list = []
for index, row in product_data.iterrows():
    if row['rate'] == 1:
        new_row = {'user_id':row['user_id'],"product" : row['product']}
        rows_list.append(new_row)

In [132]:
product_data2 = pd.DataFrame(rows_list)   

In [133]:
product_data2.head()

,user_id,product
0,3,จักรยานเสือภูเขา
1,6,จักรยานเสือภูเขา
2,10,จักรยานเสือภูเขา
3,12,จักรยานเสือภูเขา
4,15,จักรยานเสือภูเขา


In [134]:
onehot = product_data2[["user_id","product"]].pivot_table(index='user_id', columns='product', aggfunc=len)
onehot = onehot>0

In [138]:
onehot

product,Board game,Brownie อวกาศ,Lego,Omakase,PS5,Software ถูกลิขสิทธิ์,Surfskate,drone,กรรไกรตัดขนจมูก,กระเป๋า,กระเป๋า chanel,กะปิดอง,กะหรี่(ปั๊บ),กัญชง,กัญชา,ข้าวขาหมูคากิคาจักออนไลน์,ครีมกันแดด,คอร์ส bikini wax,คอร์สโบท็อกกราม,คอลลาเจน,จักรยานเสือภูเขา,ชาบู,ชุดนอนไม่ได้นอน,ชุดน้องหมา,ตาชั่ง,ต้นไม้ประดับห้อง,ถั่งเช่า,ที่ยกน้ำหนัก (Dumbell),น้องหมา,น้ำส้มดีโด้,บัตตาเลี่ยนที่ตัดผม,มะม่วงเบา,ยาดอง,หมอนข้าง,หมอนสุขภาพสำหรับคนนอนไม่หลับ,หมูแดดเดียว,หลอดไฟเปลี่ยนสีได้,ห้องน้ำแมวอัตโนมัติ,อาหารแมวคลีนเพื่อสุขภาพลดหุ่น,เกมส์ FIFA Online,เก้าอี้นวดไฟฟ้า,เครื่องดูดฝุ่นอัตโนมัติ (robot),เครื่องตรวจน้ำตาลในเลือด,เครื่องทำลายเอกสาร,เครื่องย่อยเศษอาหาร,เครื่องวัดความดัน,เนื้อย่าง,เนื้อวากิล,เม็ดมะม่วงหิมพานต์,เสื้อซีทรู,แผ่นเสียง,แพ็คเกจไทยเที่ยวไทย,แฟน ที่แปลว่าพัดลม,แม่โขง,แสงโสม,โรตี,ไดร์เป่าผม Dyson,ไพ่ยิปซี,ไฟอัจฉริยะ
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,True,False,True,True,True,True,False,False,True,True,False,False,True,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,True,False,True,False,False,False,False,False,True,False,False,False,False,True,True,True,False,False,False,True,False,False,True,False,False,False
2,False,False,False,False,False,False,False,False,True,True,False,True,True,False,False,False,True,False,False,True,False,True,True,False,True,False,True,True,False,True,False,True,False,True,True,True,False,False,False,False,False,True,True,False,False,True,False,False,True,True,False,True,True,True,True,True,False,True,False
3,True,False,True,False,False,True,True,True,False,True,False,False,True,False,False,False,True,False,False,False,True,True,True,True,True,True,False,True,True,True,True,True,False,True,True,True,True,False,False,False,False,False,True,True,False,True,True,True,True,True,False,False,True,False,False,True,False,False,True
4,True,False,True,True,False,False,False,False,True,True,False,False,True,False,True,False,True,False,False,False,False,True,False,False,True,True,False,False,False,False,True,False,True,True,False,True,False,False,False,True,True,True,False,False,False,False,True,True,True,False,False,False,True,True,True,True,False,False,False
5,True,False,False,False,False,True,False,False,False,True,True,False,True,False,False,False,True,False,True,True,False,True,False,True,False,True,False,True,True,True,False,False,False,False,True,True,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,True,False,False,True,True,True,False
6,False,False,True,False,False,False,False,False,True,True,False,False,True,False,False,False,True,False,False,True,True,True,False,True,True,True,False,True,True,False,False,True,False,True,True,True,False,False,False,False,False,True,False,False,False,True,True,True,True,True,False,False,True,False,False,True,False,False,False
7,True,False,False,False,False,True,False,False,False,True,False,True,True,False,False,False,True,False,False,False,False,True,False,True,False,True,False,True,False,True,False,True,False,True,True,True,False,False,False,False,False,False,False,False,False,True,True,True,True,False,False,False,True,True,True,True,False,False,False
8,True,False,False,False,False,True,False,False,False,True,False,False,True,False,True,True,True,False,False,False,False,True,False,False,True,True,True,True,False,False,False,False,False,True,False,True,False,False,False,False,False,True,True,False,False,True,True,True,True,False,False,True,True,False,False,True,False,True,True
9,False,False,True,False,False,True,False,False,False,True,False,False,True,False,False,False,True,False,False,True,False,True,False,True,False,True,False,True,False,False,False,True,True,True,False,True,False,False,False,False,False,False,False,False,False,True,True,True,True,False,False,False,False,False,False,True,False,False,False


In [144]:
from mlxtend.frequent_patterns import association_rules, apriori

# compute frequent items using the Apriori algorithm
frequent_itemsets = apriori(onehot, min_support = 0.5, max_len = 2, use_colnames=True)

# compute all association rules for frequent_itemsets
rules = association_rules(frequent_itemsets)
rules.shape

(174, 9)

In [145]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Board game),(Software ถูกลิขสิทธิ์),0.622222,0.800000,0.533333,0.857143,1.071429,0.035556,1.400000
1,(Board game),(กระเป๋า),0.622222,1.000000,0.622222,1.000000,1.000000,0.000000,inf
2,(Board game),(กะหรี่(ปั๊บ)),0.622222,0.955556,0.600000,0.964286,1.009136,0.005432,1.244444
3,(Board game),(ครีมกันแดด),0.622222,1.000000,0.622222,1.000000,1.000000,0.000000,inf
4,(Board game),(ชาบู),0.622222,1.000000,0.622222,1.000000,1.000000,0.000000,inf
...,...,...,...,...,...,...,...,...,...
169,(เม็ดมะม่วงหิมพานต์),(แฟน ที่แปลว่าพัดลม),0.911111,0.822222,0.777778,0.853659,1.038233,0.028642,1.214815
170,(โรตี),(เม็ดมะม่วงหิมพานต์),0.977778,0.911111,0.888889,0.909091,0.997783,-0.001975,0.977778
171,(เม็ดมะม่วงหิมพานต์),(โรตี),0.911111,0.977778,0.888889,0.975610,0.997783,-0.001975,0.911111
172,(โรตี),(แฟน ที่แปลว่าพัดลม),0.977778,0.822222,0.800000,0.818182,0.995086,-0.003951,0.977778


In [146]:
rec_1 = product_rec(1)

In [167]:
rec_1["product"]

138                              แม่โขง
183                               กัญชา
408                     เก้าอี้นวดไฟฟ้า
453                    ต้นไม้ประดับห้อง
498                            หมอนข้าง
552                             ตาชั่ง 
633                              แสงโสม
687                        กะหรี่(ปั๊บ)
723                                Lego
777                  แฟน ที่แปลว่าพัดลม
813                               ยาดอง
948     เครื่องดูดฝุ่นอัตโนมัติ (robot)
1002                         เนื้อวากิล
1047                        หมูแดดเดียว
1272                          เนื้อย่าง
1362                            กระเป๋า
1407                               ชาบู
1497                         Board game
1578                            Omakase
1632                 เม็ดมะม่วงหิมพานต์
1713                บัตตาเลี่ยนที่ตัดผม
1848                    กรรไกรตัดขนจมูก
1890                                PS5
1992                               โรตี
2057                            กะปิดอง


In [171]:
rules[rules["antecedents"] == frozenset({'Board game'})][["antecedents","consequents"]]

,antecedents,consequents
0,(Board game),(Software ถูกลิขสิทธิ์)
1,(Board game),(กระเป๋า)
2,(Board game),(กะหรี่(ปั๊บ))
3,(Board game),(ครีมกันแดด)
4,(Board game),(ชาบู)
5,(Board game),(เม็ดมะม่วงหิมพานต์)
6,(Board game),(แฟน ที่แปลว่าพัดลม)
7,(Board game),(โรตี)


In [183]:
lis = rec_1["product"].values.tolist()

In [184]:
lis

['แม่โขง',
 'กัญชา',
 'เก้าอี้นวดไฟฟ้า',
 'ต้นไม้ประดับห้อง',
 'หมอนข้าง',
 'ตาชั่ง ',
 'แสงโสม',
 'กะหรี่(ปั๊บ)',
 'Lego',
 'แฟน ที่แปลว่าพัดลม',
 'ยาดอง',
 'เครื่องดูดฝุ่นอัตโนมัติ (robot)',
 'เนื้อวากิล',
 'หมูแดดเดียว',
 'เนื้อย่าง',
 'กระเป๋า',
 'ชาบู',
 'Board game',
 'Omakase',
 'เม็ดมะม่วงหิมพานต์',
 'บัตตาเลี่ยนที่ตัดผม',
 'กรรไกรตัดขนจมูก',
 'PS5',
 'โรตี',
 'กะปิดอง',
 'แพ็คเกจไทยเที่ยวไทย',
 'เกมส์ FIFA Online',
 'น้ำส้มดีโด้',
 'Software ถูกลิขสิทธิ์',
 'ครีมกันแดด']

In [189]:
for i in lis:
    # reccom = rules[rules["antecedents"] == frozenset({i})][["antecedents","consequents"]]
    reccom = rules[rules["antecedents"] == frozenset({i})][["antecedents","consequents"]]
    if len(reccom.index) != 0:
        print(reccom)

            antecedents              consequents
25   (ต้นไม้ประดับห้อง)  (Software ถูกลิขสิทธิ์)
45   (ต้นไม้ประดับห้อง)                (กระเป๋า)
67   (ต้นไม้ประดับห้อง)           (กะหรี่(ปั๊บ))
87   (ต้นไม้ประดับห้อง)             (ครีมกันแดด)
107  (ต้นไม้ประดับห้อง)                   (ชาบู)
128  (ต้นไม้ประดับห้อง)               (หมอนข้าง)
129  (ต้นไม้ประดับห้อง)            (หมูแดดเดียว)
130  (ต้นไม้ประดับห้อง)              (เนื้อย่าง)
131  (ต้นไม้ประดับห้อง)     (เม็ดมะม่วงหิมพานต์)
132  (ต้นไม้ประดับห้อง)     (แฟน ที่แปลว่าพัดลม)
133  (ต้นไม้ประดับห้อง)                   (โรตี)
    antecedents           consequents
49   (หมอนข้าง)             (กระเป๋า)
71   (หมอนข้าง)        (กะหรี่(ปั๊บ))
91   (หมอนข้าง)          (ครีมกันแดด)
110  (หมอนข้าง)                (ชาบู)
142  (หมอนข้าง)         (หมูแดดเดียว)
145  (หมอนข้าง)  (เม็ดมะม่วงหิมพานต์)
148  (หมอนข้าง)  (แฟน ที่แปลว่าพัดลม)
150  (หมอนข้าง)                (โรตี)
    antecedents              consequents
24    (ตาชั่ง )  (Software ถู